In [ ]:
!pip install transformers pandas numpy scikit-learn tensorflow -q

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,classification_report,accuracy_score
from transformers import ElectraTokenizer,TFElectraForSequenceClassification
import os

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)


In [ ]:
def load_and_validate_data (file_path="/content/drive/My Drive/Project and Coding/BERT AND ELECTRA/stock_data.csv"):
  try:
    df=pd.read_csv(file_path)
    if 'Text' not in df.columns or 'Sentiment' not in df.columns:
      raise ValueError("Dataset must contain 'Text' and 'Sentiment' columns")
    df=df.dropna(subset=['Text','Sentiment'])
    df['Sentiment'] = df['Sentiment'].apply(lambda x: 0 if x == -1 else 1)

    print("Dataset Info:")
    print(df.info())
    print("\nSample Data:")
    print(df.head())
    print("\nLabel Distribution:")
    print(df['Sentiment'].value_counts())

    return df
  except FileNotFoundError:
    print("Error: stock_data.csv not found.")
    return None


In [ ]:
df=load_and_validate_data()
if df is None:
  raise SystemExit("Failed to load dataset. Exiting.")

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5791 entries, 0 to 5790
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       5791 non-null   object
 1   Sentiment  5791 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 90.6+ KB
None

Sample Data:
                                                Text  Sentiment
0  Kickers on my watchlist XIDE TIT SOQ PNK CPW B...          1
1  user: AAP MOVIE. 55% return for the FEA/GEED i...          1
2  user I'd be afraid to short AMZN - they are lo...          1
3                                  MNTA Over 12.00            1
4                                   OI  Over 21.37            1

Label Distribution:
Sentiment
1    3685
0    2106
Name: count, dtype: int64


In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['Text'],
    df['Sentiment'],
    test_size=0.2,
    random_state=42,
    stratify=df['Sentiment']
)


In [ ]:
tokenizer=ElectraTokenizer.from_pretrained('google/electra-small-discriminator')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def tokenize_data(texts):
  return tokenizer(
      list(texts),
      truncation=True,
      padding=True,
      max_length=128,
      return_tensors='tf'
  )
train_encoding=tokenize_data(train_texts)
test_encoding=tokenize_data(test_texts)

In [ ]:
def create_dataset(encoding,labels):
  dataset=tf.data.Dataset.from_tensor_slices((
      {
          'input_ids': encoding['input_ids'],
          'attention_mask':encoding['attention_mask']
      },
      tf.convert_to_tensor(list(labels))
  ))
  return dataset
train_dataset=create_dataset(train_encoding,train_labels)
test_dataset=create_dataset(test_encoding,test_labels)


In [ ]:
model = TFElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator', num_labels=2)

Some layers from the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5,epsilon=1e-8)
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics=[
    tf.keras.metrics.SparseCategoricalAccuracy('accuracy'),
    tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1,name='top_1_accuracy')
]
model.compile(optimizer=optimizer,loss=loss,metrics=metrics)

In [ ]:
callbacks=[
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=1,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        '/content/drive/My Drive/Project and Coding/BERT AND ELECTRA/best_electra_model',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )
]

In [ ]:
print('\n Training Electra Model...')
history=model.fit(
    train_dataset.shuffle(1000).batch(16),
    epochs=3,
    validation_data=test_dataset.batch(16),
    callbacks=callbacks
)


 Training Electra Model...
Epoch 1/3
290/290 [==============================] - 654s 2s/step - loss: 0.6288 - accuracy: 0.6585 - top_1_accuracy: 0.6585 - val_loss: 0.5691 - val_accuracy: 0.7222 - val_top_1_accuracy: 0.7222
Epoch 2/3
290/290 [==============================] - 579s 2s/step - loss: 0.5012 - accuracy: 0.7651 - top_1_accuracy: 0.7651 - val_loss: 0.4796 - val_accuracy: 0.7696 - val_top_1_accuracy: 0.7696
Epoch 3/3
290/290 [==============================] - 577s 2s/step - loss: 0.3818 - accuracy: 0.8307 - top_1_accuracy: 0.8307 - val_loss: 0.4524 - val_accuracy: 0.7955 - val_top_1_accuracy: 0.7955


In [ ]:
print("\nEvaluating model...")
predictions = model.predict(test_dataset.batch(16))
pred_labels = np.argmax(predictions.logits, axis=1)


Evaluating model...
73/73 [==============================] - 42s 519ms/step


In [ ]:
print("\nClassification Report:")
print(classification_report(test_labels, pred_labels, target_names=['Negative', 'Positive']))
print("Accuracy:", accuracy_score(test_labels, pred_labels))
print("F1 Score:", f1_score(test_labels, pred_labels))


Classification Report:
              precision    recall  f1-score   support

    Negative       0.73      0.69      0.71       421
    Positive       0.83      0.86      0.84       738

    accuracy                           0.80      1159
   macro avg       0.78      0.77      0.78      1159
weighted avg       0.79      0.80      0.79      1159

Accuracy: 0.7955133735979293
F1 Score: 0.8421052631578947


In [ ]:
save_path = "/content/drive/My Drive/Project and Coding/BERT AND ELECTRA/electra_sentiment_model"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"\nModel saved to {save_path}")


Model saved to /content/drive/My Drive/Project and Coding/BERT AND ELECTRA/electra_sentiment_model


In [ ]:
def predict_sentiment(text):
    try:
        inputs = tokenizer(
            text,
            return_tensors="tf",
            truncation=True,
            padding=True,
            max_length=128
        )

        outputs = model(inputs)
        probs = tf.nn.softmax(outputs.logits, axis=-1)
        probs_np = probs.numpy()
        pred = np.argmax(probs_np, axis=1)[0]
        confidence = probs_np[0][pred]

        return ("Positive" if pred == 1 else "Negative", confidence)
    except Exception as e:
        print(f"Error in prediction: {str(e)}")
        return None, None

In [ ]:
sample_texts = [
    "The stock market is showing strong growth this quarter",
    "Company profits are declining due to market conditions"
]

print("\nExample Predictions:")
for text in sample_texts:
    sentiment, confidence = predict_sentiment(text)
    print(f"Text: {text}")
    if sentiment is not None and confidence is not None:
        print(f"Sentiment: {sentiment}, Confidence: {confidence:.4f}\n")
    else:
        print("Error...\n")


Example Predictions:
Text: The stock market is showing strong growth this quarter
Sentiment: Positive, Confidence: 0.9754

Text: Company profits are declining due to market conditions
Sentiment: Negative, Confidence: 0.9448

